# 第6章モジュール

ここまで使ってきた関数、型、型クラスはデフォルトでimportされるPreludeというモジュールの一部

In [1]:
import Data.List

numUniques :: (Eq a) => [a] -> Int
numUniques = length . nub

In [2]:
:t nub

nub :: forall a. Eq a => [a] -> [a]

In [3]:
nub [1,2,3,4,1]

[1,2,3,4]

Hoogle ｗｗｗｗｗ  
https://www.haskell.org/hoogle/

In [4]:
:m + Data.List Data.Map Data.Set

In [7]:
import Data.List (nub, sort) -- 一部をimport

In [8]:
nub [1,2,3,4,5,1]

[1,2,3,4,5]

In [11]:
import Data.List hiding (nub) -- importから除く場合

In [10]:
nub [1,2,3,4,5,1]

Not in scope: `nub'

In [12]:
import qualified Data.Map -- こうするとData.Map.filterと使う必要がある
import qualified Data.Map as M -- こうすれば M.filterと使うことが出来る

### M.filterと関数合成演算子の違いは

`.`がqualified されたモジュール名と関数名の間に空白を開けずに置いた場合、importされた関数であるとみなされ、そうじゃなければ関数合成としてみなされる

## 標準モジュールの環数で問題を解く

In [13]:
import Data.List (words)

words "hey these are the words in this sentence"

["hey","these","are","the","words","in","this","sentence"]

In [14]:
words "hey these                   are the words in the sentence"

["hey","these","are","the","words","in","the","sentence"]

In [17]:
import Data.List (group)

group [1,1,1,1,12,2,2,2,2,23,4,4,3,3,4,34,3,5,3,1,5,6,5,89,1,1,34,4,34]

[[1,1,1,1],[12],[2,2,2,2],[23],[4,4],[3,3],[4],[34],[3],[5],[3],[1],[5],[6],[5],[89],[1,1],[34],[4],[34]]

In [18]:
import Data.List (sort)

sort [ 5,4,76,3,1,4,5,6]

[1,3,4,4,5,5,6,76]

In [25]:
import Data.Set hiding (map)
import Data.Map hiding (map)
import Data.List (map)
wordNums :: String -> [(String, Int)]
wordNums = map (\ws -> (head ws, length ws)) . D.group . D.sort . D.words

Line 2: Use &&&
Found:
\ ws -> (head ws, length ws)
Why not:
head Control.Arrow.&&& length

In [26]:
wordNums  "hey these are the words in this sentence"

[("are",1),("hey",1),("in",1),("sentence",1),("the",1),("these",1),("this",1),("words",1)]

In [27]:
import Data.List

isIn :: (Eq a) => [a] -> [a] -> Bool
needle `isIn` haystack = any (needle `isPrefixOf` ) (tails haystack)

In [28]:
"art" `isIn` "party"

True

In [29]:
[1,2] `isIn` [1,3,5]

False

In [30]:
"art" `isInfixOf` "party"

True

### import参考

https://wiki.haskell.org/Import

## シーザー暗号サラダ

次回乞うご期待

In [2]:
import Data.Char

ord 'a'

97

In [3]:
chr 97

'a'

In [4]:
encode :: Int -> String -> String
encode offset msg = map (\c -> chr $ ord c + offset) msg

In [5]:
encode 3 "hey mark"

"kh|#pdun"

In [6]:
decode :: Int -> String -> String
decode shift msg = encode (negate shift) msg

In [7]:
decode 3 "khl#pdun"

"hei mark"

In [8]:
foldl (+) 0 (replicate 100 1)

Line 1: Use sum
Found:
foldl (+) 0
Why not:
sum

100

In [9]:
foldl (+) 0 (replicate 1000000 1)

Line 1: Use sum
Found:
foldl (+) 0
Why not:
sum

1000000

# スタックオーバーフロー

- 昔バッチでやらかしたやつ

遅延評価なので、↓のようになる。

```
foldl (+) 0 [1,2,3] =
foldl (+) (0 + 1) [2,3] =
foldl (+) ((0 + 1) + 2) [3] =
foldl (+) (((0 + 1) +2) + 3) [] =
(((0 + 1) +2) + 3) =
((1 +2) + 3)
3 + 3 =
6
```

In [3]:
import Data.Char
import Data.List

digitSum :: Int -> Int
digitSum = sum . map digitToInt . show

In [12]:
:t find

find :: forall a (t :: * -> *). Foldable t => (a -> Bool) -> t a -> Maybe a

In [13]:
Nothing

Nothing

In [14]:
Just "hey"

Just "hey"

In [15]:
:t (Just "hey")

(Just "hey") :: Maybe [Char]

In [16]:
firstTo40 :: Maybe Int
firstTo40 = find (\x -> digitSum x == 40) [1..]

In [17]:
firstTo40

Just 49999

In [5]:
firstTo :: Int -> Maybe Int
firstTo n = find (\x -> digitSum x == n) [1..]

In [6]:
firstTo 10

Just 19

In [7]:
phoneBook = [("betty", "555-2938"), ("bonnie", "452-2928")]

In [8]:
findKey :: (Eq k) => k -> [(k,v)] -> v
findKey key xs = snd . head . filter (\(k,v) -> key == k) $xs

In [9]:
findKey "betty" phoneBook

"555-2938"

In [10]:
findKey "a" phoneBook

Prelude.head: empty list

In [12]:
findKey :: (Eq k) => k -> [(k,v)] -> Maybe v
findKey key [] = Nothing
findKey key ((k,v):xs)
    |key == k = Just v
    |otherwise = findKey key xs

In [13]:
findKey "a" phoneBook

Nothing

In [14]:
findKey "betty" phoneBook

Just "555-2938"

In [15]:
findKey :: (Eq k) => k -> [(k,v)] -> Maybe v
findKey key xs = foldr (\(k, v) acc -> if key == k then Just v else acc) Nothing xs

In [16]:
findKey "a" phoneBook

Nothing

# Data.Mapに潜入せよ

In [17]:
import qualified Data.Map as Map

In [18]:
Map.fromList [(3, "shoes"), (4, "trees"), (9, "bees")]

fromList [(3,"shoes"),(4,"trees"),(9,"bees")]

In [19]:
phoneBook :: Map.Map String String
phoneBook = Map.fromList $  [("betty", "555-2938"), ("bonnie", "452-2928")]

Line 2: Redundant $
Found:
Map.fromList $ [("betty", "555-2938"), ("bonnie", "452-2928")]
Why not:
Map.fromList [("betty", "555-2938"), ("bonnie", "452-2928")]

In [20]:
:t Map.lookup

Map.lookup :: forall k a. Ord k => k -> Map k a -> Maybe a

In [21]:
Map.lookup "betty" phoneBook

Just "555-2938"

In [22]:
:t Map.insert

Map.insert :: forall k a. Ord k => k -> a -> Map k a -> Map k a

In [23]:
:t Map.size

Map.size :: forall k a. Map k a -> Int

In [24]:
string2digits :: String -> [Int]
string2digits = map digitToInt . filter isDigit

In [25]:
string2digits "948-9282"

[9,4,8,9,2,8,2]

In [2]:
import Geometry

Failed to load interface for `Geometry'
Use -v to see a list of the files searched for.

In [6]:
import qualified Geometry.Sphere as Sphere
import qualified Geometry.Cuboid as Cuboid
import qualified Geometry.Cube as Cube


Failed to load interface for `Geometry.Sphere'
Use -v to see a list of the files searched for.

# なんかimportできない

> Failed to load interface for `Geometry.Sphere'

なんでや

In [5]:
import Geometry


Parse error (line 1, column 1): Unknown directive: 'cd'.

- chap6ディレクトリにGeometry.hsをおく
- chap1ディレクトリにGeometryディレクトリをおく
- Kernel再起動
- importを実行 -> Geometry.hsがないよと怒られる
- Geometry.hsをrootに戻す
- Geometryはimport 出来る
- Geometry.hogehogeはimport出来ない
